# 521153S, Deep Learning Final Project: Mini Image classification Competition with CNN

## Outline 
### In this assignment, you will learn:
* Combine all you learned from the previous assignments.
* Build your own CNN as you like with Pytorch, train and validate it on a given dataset.
* Test your CNN model on our server.

### Tasks (<span style="color:orange">40 points</span>)
We want to keep the task as clean and simple as possible. 
1. You would be given a dataset containing 45,000 grayscale 32x32 images. Also, we will set a private testing data and upload it to our server. You could only use the private testing data to test your model. Instructions about evaluating your model would be given along with the project in Moodle.
2. Before evaluating your model on the server, you have to train your model based on the given 45,000 images. Specifically, in these images, there are nine classes, and each class has 5000 images. The testing data on the server has 9000 images, and each class has 1000 images.
3. To get a good CNN model, basically, there are some rules for you to follow which would be considered when grading your report:
  * Make the best out of the given images. It means you have to split it into training and validation set, training your model on the training set and validating the trained model on the validation set. If the accuracy on the validation set is not good, then you have to adjust your CNN model structure or some adjustable parameters. For example, batch size, learning rate, momentum on SGD, lambda in weight decay, etc.  Then retrain your model until the accuracy on the validation set is good enough because the testing data would have similar accuracy with the validation set. 
  * It also means that you could do some augmentation on the training set. This includes randomly flipping, cropping a small window in a random location within the image(refer to assignment 4), adding some noise, resizing-and-cropping, etc. All these are to make the training process more tolerant. 
  * From the CNN structure perspective, you also need to design your CNN model by yourself. Well-known network architecture you can use includes [ResNet](https://arxiv.org/abs/1512.03385), [Inception](https://arxiv.org/abs/1512.00567), [VGGNet](https://arxiv.org/abs/1409.1556), [DenseNet](https://arxiv.org/abs/1608.06993), [MobileNet](https://arxiv.org/abs/1801.04381), [ShuffleNet](https://arxiv.org/abs/1807.11164), [ResNeXt](https://arxiv.org/abs/1611.05431) etc. 
4. Similar to real-life applications, your model will be tested with unknown data. In this project,  after training and validating the model, you need to test it on our hold-on testing dataset. We will provide you with a submission server and a leaderboard. The instructions would be given alongside the project in Moodle. 
5. Please give a pdf report (also your source code, e.g., this Jupyter notebook file), documenting the whole model training process and also the evaluated accuracy on the server. Tensorboard visualization is also necessary for your report to visualize your network structure, accuracy, and losses, etc. as done in assignment 4.
6. You need to return the pdf report as well as your trained model (a checkpoint file) with your source code file to moodle. We will run your model on the server and compare the results with the one written in the reports. 

### Grading
You can get 40 points in total.
  * You will get <span style="color:orange;font-weight:bold">20 points</span> if your model achieve more than 82.5% of the testing accuracy.
  * You will get <span style="color:orange;font-weight:bold">20 points</span> if your report is clear and well-organized.
  
### Files you have to submit
please submit a .zip file containing:
1. a pdf report;
2. source code files (jupyter notebook or common python files);
3. a checkpoint file (which saves your trained model).

### Group members
Maximum 2 members. 


### Environment
Python 3, Numpy, matplotlib, torch, torchvision...

### Dataset
Please follow the code below to download the 45,000 images and corresponding labels. <br>
We have already split them into training and validation set; please refer to assignment 3 and 4 to create your DataLoader, with your data augmentation methods. Good luck. 

#### Download the given dataset

In [2]:
! pip install utils 

In [5]:
# import necessary packages
import os, time
import torch 
import requests, zipfile, sys
import numpy as np
import matplotlib.pyplot as plt 
from torch.utils.data import Dataset, DataLoader
from utils import download_given_data, get_preds_figure
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, utils
import random, matplotlib
import pandas as pd
from torchvision.models.resnet import BasicBlock, ResNet
from torch.utils.tensorboard import SummaryWriter

download_given_data('./')
print(torch.cuda.is_available())


Data was already downloaded and extracted!
False


In [9]:
### GROUP INFO
### GROUP NAME: maksalaatikko
### ID: 004
### PWD: 2497530
### IMAGE LIST: 004

### IF YOU DON'T WANT TO USE GPU FOR THIS PROJECT, TOGGLE THE BOOL BELOW
use_cuda = True
#use_cuda = use_cuda and torch.cuda.is_available()

print(use_cuda)
###HYPERPARAMETERS
batch_size = 15
num_epochs = 20

lr = 0.01

#Download the model from here
#https://unioulu-my.sharepoint.com/:u:/g/personal/tmusta_student_oulu_fi/EZrVZjNykZBAigx2d84xtPIBQdTNAYJXrAj2qGmoRsyWMw

### Whether to train a new model or to test an old one. Old model requires 'model.weights' to exist in $PWD
train_model = True
test_model = True
save_path = 'model.weights'
if not train_model:
    assert os.path.isfile(save_path), "No model to be tested"

True


In [10]:
###DATASET. ACCEPTS CSV FILE AS ARGUMENT. CSV FILE SHOULD CONTAIN IMAGE PATH AND ACCORDING ANNOTATION WITH IT.

class ImageDataset(Dataset):
    def __init__(self, imagefile, transform = None):
        self.classes = []
        self.samples = []
        
        image_list = pd.read_csv(imagefile, header=None)
        self.samples = [image_list.iloc[i][0] for i in range(len(image_list))]
        self.classes = [image_list.iloc[i][1] for i in range(len(image_list))]
            
        self.transform = transform
        self.classes_n = max(self.classes)+1
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, ndx):
        sample = self.samples[ndx]
        label = None
        if self.classes[ndx] < 0:
            label = self.classes[ndx]
        else:
            label = torch.zeros((self.classes_n, 1))
            label[self.classes[ndx]] = 1
            label = torch.t(label).squeeze()
        sample = Image.open(sample)
        if self.transform:
            sample = self.transform(sample)
        return sample, label
    
    def get_classes_n(self):
        return self.classes_n

traintransform = transforms.Compose([
                                transforms.Grayscale(num_output_channels=3),
                                #transforms.Resize(256)
                                transforms.Resize(224),
                                #transforms.CenterCrop(224),
                                #transforms.RandomRotation(degrees=90),
                                transforms.RandomHorizontalFlip(0.25),
                                #transforms.RandomVerticalFlip(0.25),
                                transforms.ToTensor()
                                #transforms.RandomErasing()
                                #transforms.Normalize([0.5],[0.5])
                                    ])
valtransform = transforms.Compose([
                              transforms.Grayscale(num_output_channels=3),
                              transforms.Resize(224),
                              transforms.ToTensor()])
trainset = ImageDataset('given_data/train.csv', transform=traintransform)  
valset = ImageDataset('given_data/val.csv', transform=valtransform)
testset = None
if test_model:
    testset = ImageDataset('group_004_test.csv', transform=valtransform)

trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=64, shuffle=True)
valloader = DataLoader(valset, batch_size=batch_size, num_workers=64, shuffle=False)
testloader = None
if test_model:
    testloader = DataLoader(testset, batch_size=batch_size, num_workers=64, shuffle=False)

FileNotFoundError: [Errno 2] File b'group_004_test.csv' does not exist: b'group_004_test.csv'

In [ ]:
### MODEL

classes_n = trainset.get_classes_n()
model = ResNet(BasicBlock, [2,2,2,1])

n_inputs = model.fc.in_features

model.fc = nn.Linear(n_inputs, classes_n)


In [ ]:
### VALIDATION
def validate(loader, m, criterion):
    m.eval()
    correct = 0.0
    overall = 0.0
    loss = 0.0
    for i, data in enumerate(loader):
        sample, label = data
        if use_cuda:
            sample = sample.cuda()
            label = label.cuda()
            
        output = m(sample)
        loss += criterion(output, label).item()
        output = torch.argmax(output, dim=1)
        label = torch.argmax(label, dim=1)
        correct += sum([int(i == j) for i,j in zip(output, label)])
        overall += len(output)
    return correct / overall, loss / len(loader)

In [ ]:
### TRAINING
def train(loader, model, log='tb_graphs'):
    writer = SummaryWriter(log+'/training')
    val_writer = SummaryWriter(log+'/validation')
    if use_cuda:
        model = model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.SmoothL1Loss()
    best_acc = 0.0
    iters = len(loader)
    train_loss = 0.0
    train_acc = 0.0
    log_every = int(iters/ 10)
    for epoch in range(num_epochs):
        model.train()
        for i, data in enumerate(loader):
        
            sample, label = data
            if use_cuda:
                sample = sample.cuda()
                label = label.cuda()
            
            optimizer.zero_grad()
            output = model(sample)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            
            output = torch.argmax(output, dim=1)
            label = torch.argmax(label, dim=1)
            train_acc += sum([int(i == j) for i,j in zip(output, label)]) / batch_size
            train_loss += loss.item()
            if not i % log_every :
                print("Training", epoch, i, train_acc / log_every, train_loss / log_every)
                log_index = epoch * iters + i
                writer.add_scalar('Loss', train_loss / log_every, log_index) 
                writer.add_scalar('Accuracy', train_acc / log_every, log_index)

                
                train_loss = 0.0
                train_acc = 0.0
        acc, val_loss = validate(valloader, model, criterion)
        val_writer.add_scalar('Loss', val_loss, log_index) 
        val_writer.add_scalar('Accuracy', acc, log_index)
        print("Validation", acc)
        if acc > best_acc:
            
            best_acc = acc
            torch.save(model.state_dict(), save_path)
        elif epoch > num_epochs / 2:
        
            for g in optimizer.param_groups:
                g['lr'] *= 0.1

In [ ]:
### TEST
def test(loader, m, filename):
    preds = []
    m.eval()
    for i, data in enumerate(loader):
        sample, label = data
        if use_cuda:
            sample = sample.cuda()
            
        output = m(sample)
        output = torch.argmax(output, dim=1)
        for i in output:
            preds.append(int(i.cpu()))
    with open(filename, 'w') as f:
        for i in preds:
            f.write("%s\n" % i)

In [ ]:
### MAIN PROGRAM

if train_model:
    logdir = 'resnet2221_hflipping_smoothl1_'+str(batch_size)+'_'+str(num_epochs)
    train(trainloader, model, log=logdir)

model.load_state_dict(torch.load(save_path))
if use_cuda:
    model = model.cuda()
print("Accuracy of model: ", validate(valloader, model, nn.SmoothL1Loss())[0])
if test_model:
    test(testloader, model, 'predictions.txt')
    print("Test results created")